<a href="https://colab.research.google.com/github/miniii222/BOAZ_adv_project/blob/master/project_code/model/make_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# Google 드라이브를 마운트하려면 이 셀을 실행하세요.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.listdir('./drive/My Drive/colab/recsys/data')
path = './drive/My Drive/colab/recsys/data/'

In [0]:
import pandas as pd
import numpy as np

In [0]:
#train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path + 'test.csv')

In [0]:
print('test data에서 맞춰야되는 개수 : ',len(test[test.reference.isnull()]))
print('test data에서 clickout 개수 : ', len(test[test.action_type == 'clickout item']))

test data에서 맞춰야되는 개수 :  261731
test data에서 clickout 개수 :  528779


In [0]:
528779 - 261731

267048

In [0]:
print('test data에서의 click out 하는 session id 개수 : ',len(test[(test.action_type == 'clickout item')]['session_id'].unique()))
print('test data에서 맞춰야 되는 session id 개수 : ',len(test[test.reference.isnull()]['session_id'].unique()))

test data에서의 click out 하는 session id 개수 :  275679
test data에서 맞춰야 되는 session id 개수 :  261731


In [0]:
not_na = list(set(test[(test.action_type == 'clickout item')]['session_id'].unique()) - set(test[test.reference.isnull()]['session_id'].unique()))

In [55]:
#NA인 clickout
na_clickout = test[(test.action_type == 'clickout item') & (test.reference.isnull())]
print('전체 개수 : ',len(na_clickout))
print('session_id unique : ',len(na_clickout.session_id.unique()))

전체 개수 :  253573
session_id unique :  253573


In [56]:
na_clickout = na_clickout[['user_id','session_id','timestamp','step']]
na_clickout.reset_index(inplace = True)
na_clickout.head()

,index,user_id,session_id,timestamp,step
0,6,004A07DM0IDW,1d688ec168932,1541555799,7
1,8,009RGHI3G9A3,f05ab0de907e2,1541570940,2
2,10,00Y1Z24X8084,26b6d294d66e7,1541651823,2
3,15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5
4,61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46


In [0]:
na_clickout.tail(3)

,index,user_id,session_id,timestamp,step
253570,3782327,ZYMVSZ5A3KQI,8cd16c29b733b,1541695019,2
253571,3782333,ZZ39YE45SZIE,f5db4092ec9fc,1541551945,6
253572,3782334,ZZCM39YKI3NR,6226bde1465e7,1541601178,1


In [0]:
0~5
7~7
9~9
11~14
16~60

In [0]:
first = (np.array([-1] + list(na_clickout['index'].values))[:-1] +1)[:-1]
last = (na_clickout['index'].values - 1)[:-1]

In [0]:
fl = np.stack((first, last), axis = 1)
#fl = [range(flfl[0], flfl[1] +1) for flfl in fl]

In [0]:
test['TF'] = np.nan
i = 0
for f,l in zip(first[:63393],last[:63393]):
    i+=1
    if i%10000 == 0 :
      print(i)
    test.TF[f:l+1] = True

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


10000
20000
30000
40000
50000
60000


In [0]:
#test['TF'].to_pickle(path+ 'TF.p')

In [0]:
len(first)/4

63393.0

In [0]:
tf1 = pd.read_pickle(path + 'TF.p')
tf2 = pd.read_pickle(path + 'tf2.p')

In [0]:
tf1[63393:] = tf2[63393:]

In [0]:
test['TF'] = tf1

In [98]:
test2 = test[test.TF == True]
test2.shape

(131341, 13)

In [99]:
test2.reset_index(drop = True, inplace = True)
test2.head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,TF
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,True
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,True
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,True


In [100]:
test2.reset_index(inplace = True)
test2.head(3)

,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,TF
0,0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,True
1,1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,True
2,2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,True


In [0]:
answer = test2[test2.action_type == 'clickout item'].groupby(['user_id', 'session_id']).last().reset_index(drop = True)['index']

In [102]:
test2['answer'] = False
test2.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,TF,answer
0,0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,True,False
1,1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,True,False
2,2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,True,False


In [103]:
test2.loc[answer, 'answer'] = True

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [104]:
test2.drop(['index','TF'], axis = 1, inplace = True)
test2.to_csv(path + 'test2.csv', index = False)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [105]:
test2.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,answer
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,False
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,False
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,False
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,False
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,False
